In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np

df1 = pd.read_csv("en_ru_1m_yandex.csv")
df3 = pd.read_csv("en_ru.csv")
df4 = pd.read_csv("en_ru_2.csv")
df5 = pd.read_csv("news.csv")

df = pd.concat([df1,df3,df4,df5])
df = df[['ru', 'en']]
df = df.reset_index()
df = df.dropna()
df = df.drop_duplicates()

df['lenru'] = df.ru.apply(lambda x:len(x.split()))
df['lenen'] = df.en.apply(lambda x:len(x.split()))
df = df.loc[ (df.lenen < 60) & (df.lenru < 60)]
df.to_csv("test.csv")

In [2]:
df

,index,ru,en,lenru,lenen
0,0,такое развитие характера гарри может разочаров...,this new development in harry's character may ...,28,41
1,1,решение суда (группа вернулась под крыло к ele...,a nondisclosure clause in the final settlement...,24,24
2,2,"когда тебе 18 или 19 лет, легко перенимать бан...","when you're 18 or 19 years old, you have that ...",15,15
3,3,а сейчас куча триьютов тем же самым black sabb...,now you have black sabbath and kiss tribute al...,11,9
4,4,"я был единственным, кто занялся копированием д...",i was the one who sat down and copied them.,9,10
...,...,...,...,...,...
2511005,193207,их достижения остаются одними из самых значите...,the ability to begin to see more clearly than ...,10,27
2511006,193208,в тоже время революционное поколение зумы все ...,"forced to reckon with your own vulnerability, ...",22,31
2511007,193209,"в регионе, который преклоняется перед пожилыми...",mandela epitomized this rare gift.,21,5
2511008,193210,"трое из десяти южноафриканцев младше 15 лет, ч...",how else could he have personally invited one ...,18,24


In [4]:
import torch

# Load Data

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [6]:
import torch
from translation.models.encoder import Encoder
from translation.models.decoder import Decoder
from translation.models.transformers import Transformer

import numpy as np
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq

from translation.models.transformers import Transformer
from translation.models.encoder import Encoder
from translation.models.decoder import Decoder


def preprocess_function(batch,lang='ru-en'):
    l1 = lang.split('-')[0]
    l2 = lang.split('-')[1]
    model_inputs = tokenizer(
        batch[l1], max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch[l2], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



torch.set_float32_matmul_precision('medium')
raw_dataset = load_dataset('csv',
        data_files='test.csv')
print(raw_dataset)
lang_pair = "ru-en"
l1 = lang_pair.split('-')[0]
l2 = lang_pair.split('-')[1]
lang_pair = "ru-en"
model_checkpoint = f"Helsinki-NLP/opus-mt-{lang_pair}"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
split = raw_dataset['train'].train_test_split(test_size=0.01, seed=42)
max_input_length = 60
max_target_length = 60

tokenized_datasets = split.map(
    preprocess_function,
    batched=True,
    remove_columns=split["train"].column_names,
)


data_collator = DataCollatorForSeq2Seq(tokenizer)
batch = data_collator([tokenized_datasets["train"][i] for i in range(0, 5)])
batch.keys()


train_loader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=256,
    collate_fn=data_collator,
    num_workers=6, pin_memory=True,
)
valid_loader = DataLoader(
    tokenized_datasets["test"],
    batch_size=256,
    collate_fn=data_collator,
    num_workers=6, pin_memory=True
)



Using custom data configuration default-ffbd61ec79cfe0b0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

/usr/local/lib/python3.9/dist-packages/datasets/download/streaming_download_manager.py:695: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ffbd61ec79cfe0b0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'ru', 'en', 'lenru', 'lenen'],
        num_rows: 2480312
    })
})


/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


  0%|          | 0/2456 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [7]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2455508
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 24804
    })
})

# Load Model

In [8]:
from translation.models.transformers import Transformer
from translation.models.encoder import Encoder
from translation.models.decoder import Decoder

dkey = 64
dmodel = 512
nheads = 8
nlayers = 4
dropout = 0.1

encoder = Encoder(vocab_size=tokenizer.vocab_size + 1,
                  max_len=60,
                  d_key=64,
                  d_model=512,
                  n_heads=8,
                  n_layers=4,
                  dropout_prob=0.1)
decoder = Decoder(vocab_size=tokenizer.vocab_size + 1,
                  max_len=60,
                  d_key=64,
                  d_model=512,
                  n_heads=8,
                  n_layers=4,
                  dropout_prob=0.1)
transformer = Transformer(encoder, decoder)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
encoder.to(device)
decoder.to(device)

Using encoder without a target.
cuda:0


Decoder(
  (embedding): Embedding(62519, 512)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): DecoderBlock(
      (linear1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (linear2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (linear3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mhatt1): MultiHeadAttention(
        (key): Linear(in_features=512, out_features=512, bias=True)
        (query): Linear(in_features=512, out_features=512, bias=True)
        (value): Linear(in_features=512, out_features=512, bias=True)
        (fc): Linear(in_features=512, out_features=512, bias=True)
      )
      (mhatt2): MultiHeadAttention(
        (key): Linear(in_features=512, out_features=512, bias=True)
        (query): Linear(in_features=512, out_features=512, bias=True)
        (value): Linear(in_features=512, out_features=512, bias=True)
        (fc): Linear(in_features

In [10]:
from torchinfo import summary

summary(transformer)

Layer (type:depth-idx)                        Param #
Transformer                                   --
├─Encoder: 1-1                                --
│    └─Embedding: 2-1                         32,009,728
│    └─PositionalEncoding: 2-2                --
│    │    └─Dropout: 3-1                      --
│    └─Sequential: 2-3                        --
│    │    └─EncoderBlock: 3-2                 3,152,384
│    │    └─EncoderBlock: 3-3                 3,152,384
│    │    └─EncoderBlock: 3-4                 3,152,384
│    │    └─EncoderBlock: 3-5                 3,152,384
│    └─LayerNorm: 2-4                         1,024
├─Decoder: 1-2                                --
│    └─Embedding: 2-5                         32,009,728
│    └─PositionalEncoding: 2-6                --
│    │    └─Dropout: 3-6                      --
│    └─Sequential: 2-7                        --
│    │    └─DecoderBlock: 3-7                 4,204,032
│    │    └─DecoderBlock: 3-8                 4,204,032
│  

In [12]:
def translate(input_sentence):

    encoder_input = tokenizer(input_sentence, return_tensors='pt',max_length=max_input_length, truncation=True).to(device)

    encoder_output = encoder(encoder_input['input_ids'], encoder_input['attention_mask'])

    decoder_input_ids = torch.tensor([[ int(tokenizer.vocab_size)]], device=device)
    decoder_attn_mask = torch.ones_like(decoder_input_ids, device=device)

    for _ in range(60):
        decoder_output = decoder(
            encoder_output,
            decoder_input_ids,
            encoder_input['attention_mask'],
            decoder_attn_mask,
        )


        prediction_id = torch.argmax(decoder_output[:, -1, :], axis=-1)
        decoder_input_ids = torch.hstack((decoder_input_ids, prediction_id.view(1, 1)))
        decoder_attn_mask = torch.ones_like(decoder_input_ids)

        if prediction_id == 0:
            break
  
    translation = tokenizer.decode(decoder_input_ids[0, 1:])

    return(translation)

In [13]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.Adam(transformer.parameters(),lr=1e-4)
torch.set_float32_matmul_precision('medium')

scaler = torch.cuda.amp.GradScaler()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', threshold=0.04, verbose=True, patience=1)

# Training loop

In [14]:
from datasets import load_metric
import evaluate 
bleu_metric = evaluate.load('bleu')
#bleu_metric = load_metric("sacrebleu")
bert_metric = load_metric("bertscore")

In [15]:
len(df)

2480312

In [16]:
training_data = []

In [21]:
from datetime import datetime
torch.set_float32_matmul_precision('medium')

scaler = torch.cuda.amp.GradScaler()

def train(model, criterion, optimizer, train_loader, valid_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', threshold=0.05, threshold_mode='rel')
    for it in range(epochs):
        model.train()
        t0 = datetime.now()
        train_loss = []
        batch_count = 0
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            # initalize gradients
            optimizer.zero_grad()
            
            # get inputs
            enc_input = batch['input_ids']
            enc_mask = batch['attention_mask']
            targets = batch['labels']

            # shift targets forwards to get decoder_input
            dec_input = targets.clone().detach()
            dec_input = torch.roll(dec_input, shifts=1, dims=1)
            dec_input[:, 0] = int(tokenizer.vocab_size)

            # convert all -100 to pad token id
            dec_input = dec_input.masked_fill(
              dec_input == -100, tokenizer.pad_token_id)

            # make decoder input mask
            dec_mask = torch.ones_like(dec_input)
            dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

            # forward pass note using f16 with torch.amp
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(enc_input, dec_input, enc_mask, dec_mask)
                loss = criterion(outputs.transpose(2, 1), targets)

            # Backward prop and grad descent step
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            
            scaler.update()
            # if using 
            #scheduler.step(optimizer)
            train_loss.append(loss.item())
            batch_count += 1
            
            # Print progress
            if batch_count%200 == 0:
                print( "Progress: ", 100*batch_count/ (len(df)*0.99/256.), " %")

        train_loss = np.mean(train_loss)

        model.eval()
        test_loss = []
        
        # Evaluate the perf on validation
        for batch in valid_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            enc_input = batch['input_ids']
            enc_mask = batch['attention_mask']
            targets = batch['labels']

            dec_input = targets.clone().detach()
            dec_input = torch.roll(dec_input, shifts=1, dims=1)
            dec_input[:, 0] = int(tokenizer.vocab_size)

            dec_input = dec_input.masked_fill(
              dec_input == -100, tokenizer.pad_token_id)

            dec_mask = torch.ones_like(dec_input)
            dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

            outputs = model(enc_input, dec_input, enc_mask, dec_mask)
            loss = criterion(outputs.transpose(2, 1), targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss

        dt = datetime.now() - t0
        translations = []
        targets = []
        bertscores = []
        bleuscores = []
        torch.save(transformer.state_dict(), f"model_{it}.pt")
        
        # compute BERT and BLEU scores on 200 examples 
        for  i in range(200):
            translations.append(translate(split['test'][i][l1])[:-4])
            targets.append(split['test'][i][l2])
            score =bert_metric.compute(predictions=[translations[i]], references=[targets[i]], lang="ru")['f1']
            bertscores.append(score)
            try:
                score =bleu_metric.compute(predictions=[translations[i]], references=[[targets[i]]])['bleu']
                bleuscores.append(score)
            except:
                bleuscores.append(0)

        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Test Loss: {test_loss:.4f}, Duration: {dt}')

        print(f"Mean BERT {np.mean(bertscores)}, Mean Bleu: {np.mean(bleuscores)}" )

        training_data.append({
            'Epoch' : it+1,
            'Train_loss' : train_loss,
            'Val_loss' : test_loss,
            'Val_BERT_score' : np.mean(bertscores),
            'Val_BLEU_score': np.mean(bleuscores),
            'time per epoch' : dt,
            'd_keys' : dkey,
            'd_model' : dmodel,
            'n_heads' : nheads,
             'n_layers' :nlayers,
            'dropout_prob' :dropout
            
        })
    return train_losses, test_losses

In [ ]:
# Training loop
train_losses, test_losses = train(
    transformer, criterion, optimizer, train_loader, valid_loader, epochs=8)

In [ ]:
df = pd.DataFrame(training_data)
df.to_csv("Training_data.csv")